In [ ]:
#Library yang digunakan
import pandas as pd
from mesa import Agent, Model
from mesa.time import RandomActivation
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
#Membuat Data
jumData = 1000
np.random.seed(42) 
data = {
    'Agen_ID': range(1, jumData + 1), #Agen_ID sejumlah nilai yang sudah diinputkan
    'Umur': np.random.randint(20, 70, size=jumData),  # Umur antara 20 dan 69 tahun
    'Pendapatan': np.random.randint(1000000, 30000000, size=jumData)  # Pendapatan antara 1 juta dan 30 juta
}

df = pd.DataFrame(data)
dir=os.getcwd()
nf="mydata.xlsx"
file = os.path.join(dir, nf)
df.to_excel(file, index=False)
print(f"\nFile {nf} dengan jumlah data {jumData} telah dibuat\n")

In [ ]:
#Proses pembuatan grafik berdasarkan data yang telah dibuat
step=100

class AgentData:
    def __init__(self, agent_id, age, income):
        self.agent_id = agent_id
        self.age = age
        self.income = income

class MyAgent(Agent):
    def __init__(self, unique_id, model, data):
        super().__init__(unique_id, model)
        self.data = data

    @property
    def age(self):
        return self.data.age

    @property
    def income(self):
        return self.data.income

class MyModel(Model):
    def __init__(self, agent_data):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.steps = 0

        for data_row in agent_data:
            agent = MyAgent(data_row.agent_id, self, data_row)
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()
        self.steps += 1

    def __str__(self):
        return f"Dengan {len(self.schedule.agents)} agen dan {self.steps} langkah"

agent_data_df = pd.read_excel(file)
model = MyModel(agent_data)
for i in range(step):
    model.step()

agent_ages = [agent.age for agent in model.schedule.agents]
agent_incomes = [agent.income for agent in model.schedule.agents]
agent_data = [AgentData(row['Agen_ID'], row['Umur'], row['Pendapatan']) for idx, row in agent_data_df.iterrows()]
plt.figure(figsize=(8, 6))
plt.scatter(agent_ages, agent_incomes, alpha=0.5, color='blue')
plt.xlabel('Umur')
plt.ylabel('Pendapatan')
plt.title(f'Agent Based Modeling\nUmur vs Pendapatan\n{model}')
plt.grid(True)
plt.show()